In [ ]:
# Elasticsearch kutuphanesi kurma

```pip install elasticsearch```

In [8]:
# Import packages

from elasticsearch import Elasticsearch

In [9]:
# ElasticSearch Yapılandırma

es = Elasticsearch(HOST="http://localhost", PORT=9200)
es = Elasticsearch()

In [11]:
# Dizinin var olup olmadığına bakiyoruz

es.indices.exists(index="first_index")

True

In [12]:
# Bir dizini silmek

## Dizini silme

es.indices.delete(index="first_index")

{'acknowledged': True}

In [13]:
es.indices.exists(index="first_index")

False

In [14]:
# Veri ekleme ve alma

## Veri ekleme

doc_1 = {"city": "Ankara", "country": "Turkiye"}
doc_2 = {"city": "Sivas", "country": "Turkiye"}
doc_3 = {"city": "Kirikkale", "country": "Turkiye"}

es.index(index="cities", doc_type="places", id=1, body=doc_1)

{'_index': 'cities',
 '_type': 'places',
 '_id': '1',
 '_version': 3,
 'result': 'updated',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 6,
 '_primary_term': 3}

In [15]:
es.index(index="cities", doc_type="places", id=2, body=doc_2)

{'_index': 'cities',
 '_type': 'places',
 '_id': '2',
 '_version': 3,
 'result': 'updated',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 7,
 '_primary_term': 3}

In [16]:
es.index(index="cities", doc_type="places", id=3, body=doc_3)

{'_index': 'cities',
 '_type': 'places',
 '_id': '3',
 '_version': 3,
 'result': 'updated',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 8,
 '_primary_term': 3}

In [17]:
# Verileri alma

res = es.get(index="cities", doc_type="places", id=2)

res

{'_index': 'cities',
 '_type': 'places',
 '_id': '2',
 '_version': 3,
 '_seq_no': 7,
 '_primary_term': 3,
 'found': True,
 '_source': {'city': 'Sivas', 'country': 'Turkiye'}}

In [18]:
# _source anahtarında bulunan verileri alma

res["_source"]

{'city': 'Sivas', 'country': 'Turkiye'}

In [19]:
# Arama sorgusu ve eşleşen belgeler

## Verileri oluşturma

doc_1 = {"sentence":"COVID-19 gunumuzun en buyuk salgin hastaligi"}
doc_2 = {"sentence":"insanlar karantina da kalarak hayatlarina devam etmeye calisiyor"}

es.index(index="turkish", doc_type="sentences", id=1, body=doc_1)
es.index(index="turkish", doc_type="sentences", id=2, body=doc_2)

{'_index': 'turkish',
 '_type': 'sentences',
 '_id': '2',
 '_version': 3,
 'result': 'updated',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 6,
 '_primary_term': 2}

In [20]:
# Sorgu oluşturma

## Sorguyu eşleştirme 

body = {
    "from":0,
    "size":0,
    "query": {
        "match": {
            "sentence":"COVID-19"
        }
    }
}

res = es.search(index="turkish", body=body)
res

{'took': 1067,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

In [21]:
body = {
    "from":0,
    "size":2,
    "query": {
        "match": {
            "sentence":"COVID-19"
        }
    }
}

res = es.search(index="turkish", body=body)
res

{'took': 7,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 1.7905316,
  'hits': [{'_index': 'turkish',
    '_type': 'sentences',
    '_id': '1',
    '_score': 1.7905316,
    '_source': {'sentence': 'COVID-19 gunumuzun en buyuk salgin hastaligi'}}]}}

In [22]:
body = {
    "from":0,
    "size":2,
    "query": {
        "match": {
            "sentence":"karantina"
        }
    }
}

res = es.search(index="turkish", body=body)
res

{'took': 4,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 0.94566,
  'hits': [{'_index': 'turkish',
    '_type': 'sentences',
    '_id': '2',
    '_score': 0.94566,
    '_source': {'sentence': 'insanlar karantina da kalarak hayatlarina devam etmeye calisiyor'}}]}}

In [23]:
# match_phrase 

body = {
    "from":0,
    "size":1,
    "query": {
        "match_phrase": {
            "sentence":"karantina"
        }
    }
}

res = es.search(index="turkish", body=body)
res

{'took': 3,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 0.94566,
  'hits': [{'_index': 'turkish',
    '_type': 'sentences',
    '_id': '2',
    '_score': 0.94566,
    '_source': {'sentence': 'insanlar karantina da kalarak hayatlarina devam etmeye calisiyor'}}]}}

In [24]:
# Sorguları birleştirme

## must, must_not and should
 
body = {
    "from":0,
    "size":1,
    "query": {
        "bool": {
            "must_not": {
                "match": {
                    "sentence":"COVID-19"
                }
            },
            "should": {
                "match": {
                    "sentence": "karantina"
                }
            }
        }
    }
}

res = es.search(index="turkish", body=body)
res

{'took': 14,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 0.94566,
  'hits': [{'_index': 'turkish',
    '_type': 'sentences',
    '_id': '2',
    '_score': 0.94566,
    '_source': {'sentence': 'insanlar karantina da kalarak hayatlarina devam etmeye calisiyor'}}]}}

In [25]:
# Normal İfade Sorguları

## Verileri güncelleme

doc_1 = {"sentence":"COVID-19 gunumuzun en buyuk salgin hastaligidir."}
doc_2 = {"sentence":"insanlar karantina da kalarak hayatlarina devam etmeye calisiyor."}
doc_3 = {"sentence":"Salgin doneminde egitim uzaktan egitim olarak yapilmaktadir."}

es.index(index="turkish", doc_type="sentences", id=1, body=doc_1)

{'_index': 'turkish',
 '_type': 'sentences',
 '_id': '1',
 '_version': 4,
 'result': 'updated',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 7,
 '_primary_term': 2}

In [26]:
es.index(index="turkish", doc_type="sentences", id=2, body=doc_2)

{'_index': 'turkish',
 '_type': 'sentences',
 '_id': '2',
 '_version': 4,
 'result': 'updated',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 8,
 '_primary_term': 2}

In [27]:
es.index(index="turkish", doc_type="sentences", id=3, body=doc_3)

{'_index': 'turkish',
 '_type': 'sentences',
 '_id': '3',
 '_version': 2,
 'result': 'updated',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 9,
 '_primary_term': 2}

In [28]:
# regexp sorgusu

body = {
    "from":0,
    "size":3,
    "query": {
        "regexp": {
            "sentence":".*"
        }
    }
}

res = es.search(index="turkish", body=body)
res

{'took': 10,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 3, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'turkish',
    '_type': 'sentences',
    '_id': '1',
    '_score': 1.0,
    '_source': {'sentence': 'COVID-19 gunumuzun en buyuk salgin hastaligidir.'}},
   {'_index': 'turkish',
    '_type': 'sentences',
    '_id': '2',
    '_score': 1.0,
    '_source': {'sentence': 'insanlar karantina da kalarak hayatlarina devam etmeye calisiyor.'}},
   {'_index': 'turkish',
    '_type': 'sentences',
    '_id': '3',
    '_score': 1.0,
    '_source': {'sentence': 'Salgin doneminde egitim uzaktan egitim olarak yapilmaktadir.'}}]}}